In [1]:
import numpy as np
import pysindy as ps
import matplotlib.pyplot as plt

import sys
sys.path.append('../')
from test_data import experiment_data,add_noise,create_data_2d
from error_bounds import *

## Higher order finite differences
We want to bound the error for the derivative 1,2, and 3
For this we need to calculate the derivative of the residue and the lagrange polynomial

## Calculate the nth derivative of sympy

In [2]:
from sympy import symbols, diff, factorial, Function, prod
import numpy as np

#### Residue $r_n(x)=\frac{u^{(n+1)}(\xi (x))}{(n+1)!}\prod_{k=0}^n(x-x_k)$

In [3]:
symbol_values={}

# Define symbols
x, k = symbols('x k')
n=3
l=3
xi = Function('xi')(x)
u = Function('u')(x)

# Define the expression
expr = u.diff(x, n + 1).subs(x, xi) / factorial(n + 1) * prod(x - symbols('x_{}'.format(i)) for i in range(n + 1))

# Take the derivative with respect to x
derivative_expr = diff(expr, x,l)

# Display the result
print(f"Lagrane polynomial L_{n}:")
print(expr)

Lagrane polynomial L_3:
(x - x_0)*(x - x_1)*(x - x_2)*(x - x_3)*Derivative(u(xi(x)), (xi(x), 4))/24


In [4]:
# Define values for the symbols
x0=1
x=x0+np.arange(0,n+1)
#symbol_values = {'x_{}'.format(i): x[i] for i in range(n + 1)}
symbol_values['x'] = 'x_{}'.format(1)

# Substitute symbol values into the expression
expr_at_values = expr.subs(symbol_values)

# Substitute symbol values into the derivative expression
derivative_at_values = derivative_expr.subs(symbol_values)

# Display the results
print(f"x: {x}")
print(f"The expression at the specified values: {expr_at_values}")
print(f"The {l}th derivative at {symbol_values['x']} with n = {n}")
print(derivative_at_values)

x: [1 2 3 4]
The expression at the specified values: 0
The 3th derivative at x_1 with n = 3
(-x_0 + x_1)*(x_1 - x_2)*(x_1 - x_3)*(Derivative(u(xi(x_1)), (xi(x_1), 5))*Derivative(xi(x_1), (x_1, 2)) + Derivative(u(xi(x_1)), (xi(x_1), 6))*Derivative(xi(x_1), x_1)**2)/8 + (-x_0 + x_1)*(x_1 - x_2)*Derivative(u(xi(x_1)), (xi(x_1), 5))*Derivative(xi(x_1), x_1)/4 + (-x_0 + x_1)*(x_1 - x_3)*Derivative(u(xi(x_1)), (xi(x_1), 5))*Derivative(xi(x_1), x_1)/4 + (-x_0 + x_1)*Derivative(u(xi(x_1)), (xi(x_1), 4))/4 + (x_1 - x_2)*(x_1 - x_3)*Derivative(u(xi(x_1)), (xi(x_1), 5))*Derivative(xi(x_1), x_1)/4 + (x_1 - x_2)*Derivative(u(xi(x_1)), (xi(x_1), 4))/4 + (x_1 - x_3)*Derivative(u(xi(x_1)), (xi(x_1), 4))/4


## Derivative

#### Lagrange coefficient $L_{n,k}(x) =\prod_{i=0,i\neq k}^n \frac{x-x_i}{x_k-x_i}$

In [5]:
#Calculates the Lagrangian
def Ln_k(x, n, k):
    Ln_k_expr = prod((x - symbols(f'x_{i}')) / (symbols(f'x_{k}') - symbols(f'x_{i}')) for i in range(0,n+1) if i != k)
    return Ln_k_expr
#Calculates the residue function
def residue(x,n):
    xi = Function('xi')(x)
    u = Function('u')(x)
    expr = u.diff(x, n + 1).subs(x, xi) / factorial(n + 1) * prod(x - symbols('x_{}'.format(i)) for i in range(0,n+1))
    return expr    

In [6]:
"""
n: finite differences order
l: order of the derivative
m: we evaluate the expression at x_m
"""
def residue_derivative(m,n,l):
    x_sympol = symbols('x')
    symbol_values={}
    symbol_values['x'] = 'x_{}'.format(m)
    #Calculate derivative
    residue_expr = residue(x_sympol,n)
    derivative = diff(residue_expr,x_sympol,l)

    residue_expr_xm = residue_expr.subs(symbol_values)
    derivative_xm = derivative.subs(symbol_values)
    return derivative_xm

"""
n: finite differences order
l: order of the derivative
m: we evaluate the expression at x_m
k: number of lagrange coefficient
"""
def lagrange_derivative(m,n,k,l):
    x_sympol = symbols('x')
    symbol_values={}
    symbol_values['x'] = 'x_{}'.format(m)
    
    lagrange_expr = Ln_k(x_sympol,n, k)
    #print(lagrange_expr)
    derivative = diff(lagrange_expr,x_sympol,l)
    
    lagrange_expr_xm = lagrange_expr.subs(symbol_values)
    derivative_xm = derivative.subs(symbol_values)
    return derivative_xm

In [7]:
n,l,m,k = 3,1,1,1
res_der=residue_derivative(m,n,l)
lagrange_der=lagrange_derivative(m,n,k,l)
print(res_der)

(-x_0 + x_1)*(x_1 - x_2)*(x_1 - x_3)*Derivative(u(xi(x_1)), (xi(x_1), 4))/24


If we assume $\tilde{u}(x)=u(x)+e(x)$ with $e(x)<\epsilon$ then for $p_n(u,x) = \sum_{k=0}^nu(x_k)L_k(x) $: 
$$ | u^{(l)}(x) - p_n^{(l)}(\tilde{u},x) | \leq |r_n^{(l)}(x)| + \epsilon\sum_{k=0}^n|L_k^{(l)}(x)| $$
We call $|r_n^{(l)}(x)|$ the approximation error and $\epsilon\sum_{k=0}^n|L_k^{(l)}(x)|$ the measurement error.

In [8]:
"""
#Replaces all derivatives specified with begin_deriv and end_deriv with constants
#m is the number of the data point used for the lagrange polynomial
def replace_u_xi_derivative(m,expression, begin_deriv, end_deriv):
    x = symbols('x')
    xi = Function('xi')(x)
    u = Function('u')(x)
    replaced_expression=expression
    #Replace all derivatives in u by constant C_u{i}
    for u_lth in range(end_deriv,begin_deriv,-1):
            u_derivative = u.diff(x, u_lth).subs(x, xi)
            u_derivative = u_derivative.subs(x,m)
            replaced_expression = replaced_expression.subs(u_derivative, symbols(f"C_u{u_lth}"))

    #Replace all derivatives in xi by constant C_xi{i}
    for xi_lth in range(0,begin_deriv):
            xi_derivative = xi.diff(x, xi_lth)
            xi_derivative = xi_derivative.subs(x,symbols(f"x_{m}"))
            replaced_expression = replaced_expression.subs( xi_derivative.subs(symbols(f"x_{m}"),m), symbols(f"C_xi{xi_lth}"))  
    return replaced_expression
"""    
def replace_u_xi_derivative(m,expression, begin_deriv, end_deriv):
    h=symbols('h')
    x = symbols('x')
    xi = Function('xi')(x)
    u = Function('u')(x)
    replaced_expression=expression
    #Replace all derivatives in u by constant C_u{i}
    for u_lth in range(end_deriv,begin_deriv,-1):
            
            u_derivative = u.diff(x, u_lth).subs(x, xi)
            u_derivative = u_derivative.subs(x,symbols(f"x_{m}")).subs(symbols(f"x_{m}"),h*m)
            #print(f"{u_lth} derivatve: {u_derivative} ")
           # print(f"Before: {replaced_expression} ")
            replaced_expression = replaced_expression.subs(u_derivative, symbols(f"C_u{u_lth}"))
            #print(f"After: {replaced_expression} ")
    #Replace all derivatives in xi by constant C_xi{i}
    #print("---")
    for xi_lth in range(begin_deriv,0,-1):
            xi_derivative = xi.diff(x, xi_lth)
            xi_derivative = xi_derivative.subs(x,symbols(f"x_{m}")).subs(symbols(f"x_{m}"),h*m)
            #print(f"{xi_lth} derivatve: {xi_derivative} ")
            #print(f"Before: {replaced_expression} ")
            replaced_expression = replaced_expression.subs( xi_derivative, symbols(f"C_xi{xi_lth}")  )
            #print(f"After: {replaced_expression} ")
    return replaced_expression

In [9]:
# we only use central differences so m=order_fd/2
def get_measurement_approximation_constants(order_fd, order_derivative,m):
     #Get measurement error
    h=symbols('h')
    x=np.arange(0,order_fd+1)
    symbol_values={}
    symbol_values = {f'x_{i}': x[i]*h for i in range(order_fd+1)}
    C_meas=0.0
    for k in range(order_fd+1):
            lagrange_der=lagrange_derivative(m,order_fd,k,order_derivative)
            value = lagrange_der.subs(symbol_values)
            C_meas += abs(value)

    #Calculate approximation error
    symbol_values = {f'x_{i}': h*x[i] for i in range(order_fd+1)}
    res_expr=residue_derivative(m,order_fd,order_derivative)
    res_der =res_expr.subs(symbol_values)
    res_expr_replaced = replace_u_xi_derivative(m,res_der, order_fd,order_fd+order_derivative+1)
    # Take the absolute value of each term individually
    C_app = sum(abs(term) for term in res_expr_replaced.as_ordered_terms())
    return C_meas,C_app

In [10]:
def upper_bound_central_differences(eps,order_fd,order_derivative,Cu,Cxi,h):
    assert(order_fd%2==0)
    m=int(order_fd/2)
    C_meas,C_app = get_measurement_approximation_constants(order_fd,order_derivative,m)
    #print(C_app,abs(C_meas))
    symbol_values = {f'C_u{i}': Cu for i in range(order_fd+1,order_fd+order_derivative+1)}
    symbol_values.update({f'C_xi{i}': Cxi for i in range(1,order_fd)})
   # print(symbol_values)
    C_app=C_app.subs(symbol_values)
    #print(C_app,C_meas)
    C_app=C_app.subs(symbols('h'),h)
    C_meas=C_meas.subs(symbols('h'),h)
    #print(C_app,C_meas)
    upper_bound = C_app+C_meas
    return upper_bound

In [11]:
order_fd,order_derivative=4,1
eps=1
Cu,Cxi=1,1
u=upper_bound_central_differences(eps,order_fd,order_derivative,Cu,Cxi,h=0.01)
u

150.000000000333